In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install geopy
import pandas as pd
from sklearn.cluster import DBSCAN
from geopy import distance
from shapely.geometry import MultiPoint
import numpy as np
from sklearn.cluster import KMeans
def distance_matrix_geo(x,y):
    '''
    compute distance matrix given longitude and latitude data
    unoptimized version, can be very slow!!
    :param x:
    :param y:
    :return:
    '''
    x = np.asarray(x)
    y = np.asarray(y)
    m,_ = x.shape
    n, _ = y.shape

    result = np.empty((m,n),dtype=float)
    for i, cord_x in enumerate(x):
        for j, cord_y in enumerate(y):
            result[i][j] = distance.great_circle(cord_x,cord_y).meters

    return result


In [ ]:
import geopandas as gpd

# Read the GeoJSON file using geopandas
# geojson_path = '/content/drive/MyDrive/Data/Politiezones.json'
geojson_path = '/content/drive/MyDrive/Datathon_Data/Processed/Belgium.provinces.WGS84.geojson'
gdf = gpd.read_file(geojson_path)

In [ ]:
gdf

,RecId,Version,AdPrKey,AdReKey,AdCoKey,NameFRE,NameDUT,NameGER,FiscSitId,UpdDate,Stat_Area,Shape_Leng,Shape_Area,geometry
0,301,1,04000,04000,01000,Région de Bruxelles-Capitale,Brussels Hoofdstedelijk Gewest,Region Brüssel-Hauptstadt,5,2020-03-26 00:00:00+00:00,16242,72162.139042,1.624198e+08,"POLYGON ((4.31176 50.79743, 4.33110 50.77551, ..."
1,302,1,10000,02000,01000,Province d’Anvers,Provincie Antwerpen,Provinz Antwerpen,5,2020-03-26 00:00:00+00:00,287612,409230.640917,2.876120e+09,"MULTIPOLYGON (((5.23796 51.26127, 5.22609 51.2..."
2,303,1,20001,02000,01000,Province du Brabant flamand,Provincie Vlaams-Brabant,Provinz Flämisch-Brabant,5,2020-03-26 00:00:00+00:00,211836,508851.938964,2.118360e+09,"POLYGON ((5.10360 50.70921, 5.10788 50.71839, ..."
3,304,1,20002,03000,01000,Province du Brabant wallon,Provincie Waals-Brabant,Provinz Wallonisch-Brabant,5,2020-03-26 00:00:00+00:00,109718,285898.405104,1.097184e+09,"POLYGON ((5.01955 50.75072, 5.00348 50.76604, ..."
4,305,1,30000,02000,01000,Province de Flandre occidentale,Provincie West-Vlaanderen,Provinz Westflandern,5,2020-03-26 00:00:00+00:00,319658,357083.649169,3.196576e+09,"POLYGON ((3.45892 50.76469, 3.48862 50.78600, ..."
5,306,1,40000,02000,01000,Province de Flandre orientale,Provincie Oost-Vlaanderen,Provinz Ostflandern,5,2020-03-26 00:00:00+00:00,300706,404804.243717,3.007064e+09,"POLYGON ((4.24215 51.35401, 4.16610 51.29283, ..."
6,307,1,50000,03000,01000,Province de Hainaut,Provincie Henegouwen,Provinz Hennegau,5,2020-03-26 00:00:00+00:00,381283,679704.608795,3.812832e+09,"MULTIPOLYGON (((3.17814 50.75616, 3.18736 50.7..."
7,308,1,60000,03000,01000,Province de Liège,Provincie Luik,Provinz Lüttich,5,2020-03-26 00:00:00+00:00,385710,574836.392063,3.857105e+09,"POLYGON ((5.39302 50.37941, 5.39558 50.38374, ..."
8,309,1,70000,02000,01000,Province de Limbourg,Provincie Limburg,Provincie Limburg,5,2020-03-26 00:00:00+00:00,242742,398314.827791,2.427425e+09,"MULTIPOLYGON (((5.68796 50.81200, 5.67841 50.8..."
9,310,1,80000,03000,01000,Province de Luxembourg,Provincie Luxemburg,Provinz Luxembourg,5,2020-03-26 00:00:00+00:00,445926,542458.940272,4.459264e+09,"POLYGON ((4.96864 49.80156, 4.99836 49.79956, ..."


In [ ]:
from shapely.geometry import Point
def return_province(lat, lon, gdf):
  point = Point(lon, lat)
  result = gdf[gdf.geometry.contains(point)]
  if not result.empty:
    province_name = result.iloc[0,5]  # Replace 'province_name' with the actual column name, replace "5" with column index
    return province_name
  else:
    return False

#df_aed['test'] = df_aed.apply(lambda row: return_province(row['lat'], row['lon'], gdf), axis=1)
def add_province_column(df, gdf):
    """
    Function to add a 'province' column to the DataFrame based on latitude and longitude coordinates.

    Parameters:
        df (DataFrame): DataFrame containing latitude and longitude columns.
        gdf (GeoDataFrame): GeoDataFrame containing province polygons with a 'geometry' column.

    Returns:
        DataFrame: DataFrame with the 'province' column added.
        int: Number of rows with invalid geometry.
    """
    province_list = []
    invalid_geometry_count = 0
    for index, row in df.iterrows():
        try:
            province = return_province(row['lat'], row['lon'], gdf)
            province_list.append(province)
        except Exception as e:
            province_list.append(None)  # Append None when an error occurs
            invalid_geometry_count += 1

    df['new_province'] = province_list
    return df, invalid_geometry_count



In [ ]:
df_aed = pd.read_csv('/content/drive/MyDrive/Datathon_Data/Processed/merged_aed_loc_with_province.csv',encoding='utf-8')

In [ ]:
df_aed

,id_x,type,address,number,postal_code,municipality,province,location,public,available,hours,full_address,new_id,id_y,lat,lon
0,13.0,NaN,Blvd. Fr. Roosevelt,24,7060,SOIGNIES,Province de Hainaut,NaN,Y,NaN,NaN,"Blvd. Fr. Roosevelt, 24, 7060, SOIGNIES, Hainaut",1,13.0,50.576765,4.065502
1,70.0,NaN,Ch. De Wégimont,76,4630,Ayeneux,NaN,NaN,NaN,NaN,NaN,"Ch. De Wégimont, 76, 4630, Ayeneux, Liège",2,70.0,50.610570,5.739783
2,71.0,NaN,Place Saint - Lambert,-1,4020,Liège,NaN,NaN,NaN,NaN,NaN,"Place Saint - Lambert, 4020, Liège, Liège",3,71.0,50.645622,5.573620
3,72.0,NaN,Rue du Doyard,-1,4990,Lierneux,NaN,NaN,NaN,NaN,NaN,"Rue du Doyard, 4990, Lierneux, Liège",4,72.0,50.287416,5.786325
4,73.0,NaN,Fond Saint Servais,-1,4000,Liège,NaN,NaN,NaN,NaN,NaN,"Fond Saint Servais, 4000, Liège, Liège",5,73.0,50.646806,5.571031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15190,16662.0,Appareil fixe-Vast apparaat,Nekkerspoel-Borcht,19,2800,Mechelen,Province d’Anvers,reddersgebouw aan zwemvijver,Non-Nee,Non-Nee,NaN,"Nekkerspoel-Borcht, 19, 2800, Mechelen, Antwerpen",15223,16662.0,51.021656,4.502027
15191,16664.0,Appareil fixe-Vast apparaat,Nieuwe Dreef,17,9160,Lokeren,Province de Flandre orientale,Locatie : ingang poort hoofdgebouw,Oui-Ja,Oui-Ja,NaN,"Nieuwe Dreef , 17, 9160, Lokeren, Oost-Vlaanderen",15224,16664.0,51.097923,3.911422
15192,16665.0,Appareil fixe-Vast apparaat,Panterschipstraat,207,9000,Gent,Province de Flandre orientale,,Oui-Ja,Non-Nee,NaN,"Panterschipstraat , 207, 9000, Gent, Oost-Vlaa...",15225,16665.0,51.095064,3.726934
15193,16666.0,Appareil fixe-Vast apparaat,Albert Leemansplein,20,1050,Bruxelles,Région de Bruxelles-Capitale,,Oui-Ja,Non-Nee,NaN,"Albert Leemansplein, 20, 1050, Bruxelles, Brux...",15226,16666.0,50.821705,4.362949


In [ ]:
new_aed_df,unvalid_count = add_province_column(df_aed, gdf)

In [ ]:
unvalid_count

2152

In [ ]:
new_aed_df.to_csv('/content/drive/MyDrive/Datathon_Data/Processed/aed_df_with_province.csv',encoding='utf-8')

In [ ]:
import pandas as pd
new_aed_df = pd.read_csv('/content/drive/MyDrive/Datathon_Data/Processed/aed_df_with_province.csv',encoding='utf-8')

In [ ]:
aed_df_bra = new_aed_df[new_aed_df['new_province']=='Province du Brabant flamand']

In [ ]:
aed_df_bra

,Unnamed: 0,id_x,type,address,number,postal_code,municipality,province,location,public,available,hours,full_address,new_id,id_y,lat,lon,new_province
13,13,95.0,NaN,Peperstraat,2,3300,TIENEN,Province du Brabant flamand,NaN,Y,NaN,NaN,"Peperstraat, 2, 3300, TIENEN, Vlaams-Brabant",14,95.0,50.806822,4.937392,Province du Brabant flamand
20,20,111.0,NaN,Hermeslaan,2,1831,MachelenBt),Province du Brabant flamand,NaN,NaN,NaN,NaN,"Hermeslaan, 2, 1831, MachelenBt), Vlaams-Brabant",21,111.0,50.881256,4.442293,Province du Brabant flamand
27,27,167.0,NaN,Vanhalstraat,1,1500,Halle,Province du Brabant flamand,NaN,y,NaN,NaN,"Vanhalstraat, 1, 1500, Halle, Vlaams-Brabant",28,167.0,50.732485,4.234913,Province du Brabant flamand
53,53,275.0,NaN,Da Vincilaan,-1,1930,Zaventem,Province du Brabant flamand,NaN,NaN,NaN,NaN,"Da Vincilaan, 1930, Zaventem, Vlaams-Brabant",54,275.0,50.889225,4.459170,Province du Brabant flamand
63,63,318.0,NaN,Interleuvenlaan,58,3001,Leuven,Province du Brabant flamand,NaN,NaN,NaN,NaN,"Interleuvenlaan , 58, 3001, Leuven, Vlaams-Bra...",64,318.0,50.847642,4.737563,Province du Brabant flamand
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15139,15139,16611.0,Appareil fixe-Vast apparaat,Torleylaan,100,1654,Huizingen,Province du Brabant flamand,In kast in de inkomhal van het gebouw Wauwerin...,Non-Nee,Non-Nee,Enkel beschikbaar tijdens de openingsuren van ...,"Torleylaan, 100, 1654, Huizingen, Vlaams-Brabant",15172,16611.0,50.746570,4.273690,Province du Brabant flamand
15146,15146,16617.0,Appareil fixe-Vast apparaat,Nijvelsebaan,190,3090,Overijse,Province du Brabant flamand,Achterkant buiten,Non-Nee,Non-Nee,07h30 - 18h,"Nijvelsebaan, 190, 3090, Overijse, Vlaams-Brabant",15179,16617.0,50.770862,4.563086,Province du Brabant flamand
15159,15159,16628.0,Appareil fixe-Vast apparaat,Reppelsebaan,65,3294,Diest,Province du Brabant flamand,,Oui-Ja,Non-Nee,NaN,"Reppelsebaan, 65, 3294, Diest, Vlaams-Brabant",15192,16628.0,51.011220,5.010322,Province du Brabant flamand
15166,15166,16635.0,Appareil fixe-Vast apparaat,Astridlaan,85,1970,Wezembeek-Oppem,Province du Brabant flamand,Buiten aan de cafetaria,Oui-Ja,Oui-Ja,NaN,"Astridlaan , 85, 1970, Wezembeek-Oppem, Vlaams...",15199,16635.0,50.833949,4.498610,Province du Brabant flamand


In [ ]:
interventions_within_aedhop = pd.read_csv('/content/drive/MyDrive/Datathon_Data/Processed/intervention_within_aed_hospital.csv',encoding='utf-8')
interventions_without_aedhop = pd.read_csv('/content/drive/MyDrive/Datathon_Data/Processed/intervention_without_aed_hospital.csv',encoding='utf-8')
interventions = pd.concat([interventions_within_aedhop,interventions_without_aedhop],axis=0)
interventions = interventions[interventions['province']=='Province du Brabant flamand']
interventions = interventions.dropna(subset=['duration_t3-t0'])
interventions = interventions.drop_duplicates(subset=['mission_id'])
interventions

,mission_id,t0_date,duration_t3-t0,eventtype_code,eventlevel,province,postalcode_intervention,lat,lon,abandon_reason
0,21221540239,2022-06-03,50.116667,P003,N1,Province du Brabant flamand,NaN,50.842169,4.469844,NaN
2,21221540463,2022-06-03,28.583333,P003,N1,Province du Brabant flamand,NaN,50.810332,4.232565,NaN
6,21221560159,2022-06-05,18.466667,P003,N1,Province du Brabant flamand,NaN,50.908946,4.203203,NaN
8,21221560745,2022-06-05,9.933333,P003,N1,Province du Brabant flamand,NaN,50.864708,4.442090,NaN
10,21221580059,2022-06-07,11.200000,P003,N1,Province du Brabant flamand,NaN,50.861866,4.708210,NaN
...,...,...,...,...,...,...,...,...,...,...
841,21231510617,2023-05-31,18.133333,P003,N1,Province du Brabant flamand,NaN,51.026739,4.974698,NaN
2487,20222770049,2022-10-04,8.231153,P003,N1,Province du Brabant flamand,1970.0,50.844530,4.494600,NaN
2493,20223400163,2022-12-06,9.383939,P003,N1,Province du Brabant flamand,1780.0,50.910190,4.296190,NaN
5632,22223120052,2022-11-08,13.666983,P003,N1,Province du Brabant flamand,NaN,50.753580,4.955580,NaN


In [ ]:
x= np.array(interventions[['lat','lon']])
x

array([[50.842169,  4.469844],
       [50.810332,  4.232565],
       [50.908946,  4.203203],
       ...,
       [50.91019 ,  4.29619 ],
       [50.75358 ,  4.95558 ],
       [50.71591 ,  4.1089  ]])

In [ ]:
y = np.array(aed_df_bra[['lat','lon']])
y

array([[50.8068221,  4.9373925],
       [50.8812562,  4.4422935],
       [50.7324848,  4.2349129],
       ...,
       [51.0112196,  5.0103219],
       [50.8339487,  4.4986099],
       [50.971477 ,  4.168257 ]])

In [ ]:
from geopy import distance
distance_matrix = distance_matrix_geo(x,y)

In [ ]:
min_distance_matrix = np.min(distance_matrix,axis=1)

In [ ]:
sum(min_distance_matrix<200)/len(min_distance_matrix)

0.22432859399684044